## Trabajo Visualización de datos UDD:

Objetivos:


Alumnos:

| Alumno | Email |
|--------|-------|
|Carolina Hoffman | chofmman@udd.cl |
|Wilder Prado     | wilderhp@gmail.com |
|Effry Vigorena   | effryvg@gmail.com |
|Felipe Guzmán    | felipeguzmanv@gmail.com|





In [30]:
## Librerias

import numpy as np
import pandas as pd
#import geopandas as gpd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
from matplotlib.patches import ConnectionPatch
from matplotlib.text import OffsetFrom
import matplotlib.image as image



import plotly as px
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from plotly.subplots import make_subplots


import warnings
warnings.filterwarnings('ignore')




II. Visualización sunburst

In [31]:
## Funciones a ocupar

def update_treemap(zona):
  global df_filter_T
 
  if zona !="ALL":
    df = df_filter_T.query(f"MACROZONA == '{zona.upper()}'")
  else:
    df = df_filter_T

  val='RESULTADO_VAL'
  txt="GRAFICO TREEMAP - SIZE VITAMINA D"

  fig = px.treemap(df, values=val,path = [px.Constant(txt),'AÑO','MACROZONA','ESTACIONES_AÑO','VIT_D_TXT'],
                color='VIT_D_TXT', 
                color_discrete_map={'Deficiente':'#ce281f',
                                     'Deseable':'#8eef44'}                
                 )
  cm = {txt:"#f2f6fa"}

  fig.for_each_trace(
    lambda t: t.update(
        marker_colors=[
            cm[id.split("/")[0]] if len(id.split("/")) == 1 else c
            for c, id in zip(t.marker.colors, t.ids)
          ]        
      )
    )
  
  cm = {
    "2018":"#ec722e",
    "2019":"#7dc343",
    "2020":"#64a2d8",
    "2021":"#eed842"
  }

  fig.for_each_trace(
    lambda t: t.update(
        marker_colors=[
            cm[id.split("/")[1]] if len(id.split("/")) == 2 else c
            for c, id in zip(t.marker.colors, t.ids)
          ]        
      )
    )

  cm = {
    "NORTE": "#e6dc77",
    "CENTRO": "#b8d6b1",
    "SUR": "#64aee5"
  }

  fig.for_each_trace(
    lambda t: t.update(
        marker_colors=[
            cm[id.split("/")[2]] if len(id.split("/")) == 3 else c
            for c, id in zip(t.marker.colors, t.ids)
          ]        
      )
    )

  cm = {
    "PRIMAVERA": "Green",
    "VERANO": "#F5761A",
    "OTOÑO": "Brown",
    "INVIERNO": "#6082B6"
  }
  fig.for_each_trace(
    lambda t: t.update(
        marker_colors=[
            cm[id.split("/")[3]] if len(id.split("/")) == 4 else c
            for c, id in zip(t.marker.colors, t.ids)
          ]        
      )
    )

  fig.update_layout(margin = dict(t=25, l=25, r=25, b=25)) 

  fig.update_layout(   
    template="plotly_white",
    height = 900
  )

  fig.update_traces(textinfo="label+percent parent") 
  fig.data[0]['textfont']['size'] = 14
  fig.show() 

def update_sunburts(year):
  global df_filter_R
  
  val='RESULTADO_VAL'
  txt="GRAFICO TREEMAP - SIZE VITAMINA D"
    
  if year !="ALL":
    df = df_filter_R.query(f"AÑO == {year}")
  else:
    df = df_filter_R
    
  fig2 = px.sunburst(df, path=['AÑO',"VIT_D_TXT", 'ADULTO_NIÑO'], values=val)
  
  colores={"Adulto":"#caccd1",
  "Niño":"#00a4e4",
  "Deficiente":"#ce281f",
  "Deseable":"#8eef44",
  "2018":"#ec722e",
  "2019":"#7dc343",
  "2020":"#64a2d8",
  "2021":"#eed842"}

  for t in fig2.data:
    lista = t.labels
    lista_p = t.parents

  colors = []

  for p in range(0,len(lista)):
    if len(lista_p[p].split("/"))==2:
      colors.append(colores[lista_p[p].split("/")[1]])
    else: 
      colors.append(colores[lista[p]])

  fig = go.Figure(go.Sunburst(
        labels=fig2['data'][0]['labels'],
        parents=fig2['data'][0]['parents'],
        values=4*fig2['data'][0]['values'],
        ids=fig2['data'][0]['ids'],
        branchvalues='total',
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}:</b> <br> Total : %{value} <extra></extra>',
        maxdepth=3
      ))
  
  fig.update_layout(   
    template="plotly_white",
        title={
        'text': "GRAFICO SUNBURTS DISTRIBUCION VITAMINA D"
        },
    height = 900     
    )
  fig.data[0]['textfont']['size'] = 14
  fig.update_traces(textinfo="label+percent parent") 

  fig.show() 

def agregar_proporcion(grupo):    
    grupo["PROP"] = 100 * grupo['VITAMINA D'] / grupo['VITAMINA D'].sum()
    return grupo
    
def condicion(x):
    x=int(x)
    if x<30:
        return "Deficiente"
    else:
        return "Deseable"

In [32]:
data = pd.read_csv("https://raw.githubusercontent.com/wilderhp/VitaminaD/main/Vitamina_D_Visualizacion_26092022.csv",sep=';', decimal=',')
data.columns = data.columns.str.upper()
data['RESULTADO_VAL']= 1
data['VIT_D_TXT'] = data['VITAMINA D'].apply(condicion)

df_filter = data[['AÑO','MACROZONA','ESTACIONES_AÑO','ADULTO_NIÑO','VIT_D_TXT','RESULTADO_VAL','VITAMINA D']]

df_filter_T = df_filter.groupby(['AÑO','MACROZONA','ESTACIONES_AÑO']).apply(agregar_proporcion)

df_filter_T

,AÑO,MACROZONA,ESTACIONES_AÑO,ADULTO_NIÑO,VIT_D_TXT,RESULTADO_VAL,VITAMINA D,PROP
0,2018,CENTRO,VERANO,Niño,Deficiente,1,22.87,0.018206
1,2018,CENTRO,VERANO,Adulto,Deficiente,1,15.04,0.011973
2,2018,CENTRO,VERANO,Adulto,Deficiente,1,17.17,0.013668
3,2018,CENTRO,VERANO,Adulto,Deficiente,1,14.44,0.011495
4,2018,NORTE,VERANO,Adulto,Deficiente,1,15.52,0.110638
...,...,...,...,...,...,...,...,...
212521,2021,CENTRO,PRIMAVERA,Adulto,Deficiente,1,6.62,0.001980
212522,2021,CENTRO,PRIMAVERA,Adulto,Deficiente,1,14.53,0.004346
212523,2021,CENTRO,PRIMAVERA,Niño,Deficiente,1,6.61,0.001977
212524,2021,CENTRO,PRIMAVERA,Adulto,Deficiente,1,18.50,0.005533


In [33]:
df_filter_T.groupby(['AÑO','MACROZONA','ESTACIONES_AÑO','VIT_D_TXT']).PROP.sum()

AÑO   MACROZONA  ESTACIONES_AÑO  VIT_D_TXT 
2018  CENTRO     INVIERNO        Deficiente    82.218896
                                 Deseable      17.781104
                 OTOÑO           Deficiente    63.508004
                                 Deseable      36.491996
                 PRIMAVERA       Deficiente    75.997636
                                                 ...    
2021  SUR        OTOÑO           Deseable      20.495784
                 PRIMAVERA       Deficiente    75.748868
                                 Deseable      24.251132
                 VERANO          Deficiente    72.476285
                                 Deseable      27.523715
Name: PROP, Length: 96, dtype: float64

In [34]:
df_filter_T.groupby(['AÑO','MACROZONA','ESTACIONES_AÑO','VIT_D_TXT']).RESULTADO_VAL.count()

AÑO   MACROZONA  ESTACIONES_AÑO  VIT_D_TXT 
2018  CENTRO     INVIERNO        Deficiente    5286
                                 Deseable       423
                 OTOÑO           Deficiente    4229
                                 Deseable      1184
                 PRIMAVERA       Deficiente    5869
                                               ... 
2021  SUR        OTOÑO           Deseable       466
                 PRIMAVERA       Deficiente    5084
                                 Deseable       705
                 VERANO          Deficiente    3191
                                 Deseable       596
Name: RESULTADO_VAL, Length: 96, dtype: int64

In [35]:
df_filter = data[['AÑO','ADULTO_NIÑO','VIT_D_TXT','RESULTADO_VAL','VITAMINA D']]

df_filter_R = df_filter.groupby(['AÑO','VIT_D_TXT']).apply(agregar_proporcion)

df_filter_R

,AÑO,ADULTO_NIÑO,VIT_D_TXT,RESULTADO_VAL,VITAMINA D,PROP
0,2018,Niño,Deficiente,1,22.87,0.005400
1,2018,Adulto,Deficiente,1,15.04,0.003551
2,2018,Adulto,Deficiente,1,17.17,0.004054
3,2018,Adulto,Deficiente,1,14.44,0.003410
4,2018,Adulto,Deficiente,1,15.52,0.003665
...,...,...,...,...,...,...
212521,2021,Adulto,Deficiente,1,6.62,0.000485
212522,2021,Adulto,Deficiente,1,14.53,0.001065
212523,2021,Niño,Deficiente,1,6.61,0.000485
212524,2021,Adulto,Deficiente,1,18.50,0.001356


In [36]:
df_filter_R.groupby(['AÑO','VIT_D_TXT','ADULTO_NIÑO']).PROP.sum()

AÑO   VIT_D_TXT   ADULTO_NIÑO
2018  Deficiente  Adulto         92.696259
                  Niño            7.303741
      Deseable    Adulto         89.502991
                  Niño           10.497009
2019  Deficiente  Adulto         92.488915
                  Niño            7.511085
      Deseable    Adulto         90.268054
                  Niño            9.731946
2020  Deficiente  Adulto         91.958981
                  Niño            8.041019
      Deseable    Adulto         91.577155
                  Niño            8.422845
2021  Deficiente  Adulto         90.163466
                  Niño            9.836534
      Deseable    Adulto         91.133852
                  Niño            8.866148
Name: PROP, dtype: float64

In [37]:
df_filter_R.groupby(['AÑO','VIT_D_TXT','ADULTO_NIÑO']).RESULTADO_VAL.count()

AÑO   VIT_D_TXT   ADULTO_NIÑO
2018  Deficiente  Adulto         22681
                  Niño            1630
      Deseable    Adulto          4032
                  Niño             465
2019  Deficiente  Adulto         33416
                  Niño            2534
      Deseable    Adulto          8263
                  Niño             885
2020  Deficiente  Adulto         37161
                  Niño            3183
      Deseable    Adulto          7553
                  Niño             712
2021  Deficiente  Adulto         70443
                  Niño            7873
      Deseable    Adulto         10680
                  Niño            1015
Name: RESULTADO_VAL, dtype: int64

In [38]:
#Treemap
dropdown_zona = widgets.Dropdown(
    value='ALL', 
    options=['ALL','Norte', 'Centro', 'Sur'], 
    description='Zona :'
)

widgets.interact(update_treemap, zona=dropdown_zona) 

interactive(children=(Dropdown(description='Zona :', options=('ALL', 'Norte', 'Centro', 'Sur'), value='ALL'), …

<function __main__.update_treemap(zona)>

In [39]:
#Sunburts
dropdown_año = widgets.Dropdown(
    value='ALL', 
    options=['ALL','2018', '2019', '2020', '2021'], 
    description='Año :'
)

widgets.interact(update_sunburts, year = dropdown_año) 

interactive(children=(Dropdown(description='Año :', options=('ALL', '2018', '2019', '2020', '2021'), value='AL…

<function __main__.update_sunburts(year)>